In [1]:
import numpy as np
import torch
from sympy.utilities.iterables import subsets
from sympy.utilities.iterables import multiset_permutations
device = 'cuda' if torch.cuda.is_available else 'cpu'

%load_ext line_profiler

In [2]:
class Game:
    
    def __init__(self, player1, player2, size=3, n_dim=2):
        assert(type(n_dim) is int and n_dim >= 2), "wrong n_dim"
        assert(type(size) is int and size >= 2), "wrong size"
        self.n_dim = n_dim
        self.size = size
        self.player1 = player1
        self.player2 = player2
        self.board = np.zeros([size]*n_dim, dtype=int)
    
    def score(self):
        score_p1 = 0
        score_p2 = 0
        
        def slice_to_mask(L):
            """
            Enables to use slicing operator like array[x, y, :, z] with choosing the position
            of the symbol ':' (represented with a -1 instead). For example L can be equal to
            [0, 0, -1, 0] if we want to access self.board[0, 0, :, 0]
            """
            mask = np.zeros([self.size] * self.n_dim, dtype=bool)
            dim = L.index(-1)
            for tile in range(self.size):
                L[dim] = tile
                mask[tuple(L)] = True
            return mask
        
        # vertical and horizontal axis
        all_axis = []
        for d in range(self.size ** self.n_dim):
            all_axis.append([(d // self.size**k) % self.size for k in range(self.n_dim)[::-1]])
            # example in 3D case with size 3 :
            # all_axis = [ [i, j, k] for i = 0, 1, 2 for j = 0, 1, 2 for k = 0, 1, 2 ]
        for d in range(self.n_dim):
            d_axis = np.array(all_axis)
            d_axis[:, d] = -1
            d_axis = np.unique(d_axis, axis=0)
            for axis in d_axis:
                space_mask = slice_to_mask(list(axis))
                in_game_axis = self.board[space_mask]
                axis_value = in_game_axis.sum().item()
                if axis_value == self.size:
                    score_p1 += 1
                elif axis_value == -self.size:
                    score_p2 += 1
        
        # diagonal axis
        diag = np.array([range(self.size)]).T
        antidiag = np.array([range(self.size-1, -1, -1)]).T
        poss_diag = np.array([diag, antidiag])
        poss_index = list(range(self.size))
        coords_to_check = set()
        for dof in range(self.n_dim-2, -1, -1):
            dof_fc = self.n_dim - dof
            cpt = 0
            for fc in subsets(poss_diag, dof_fc, repetition=True):
                if cpt == int(dof_fc / 2) + 1:
                    break
                cpt += 1
                frozen_comp = np.array(fc).reshape((dof_fc, self.size)).T
                if dof > 0:
                    for free_comp in subsets(poss_index, dof, repetition=True):
                        free_comp_array = np.repeat(np.array([free_comp]), self.size, axis=0)
                        coords = np.hstack((free_comp_array, frozen_comp))
                        for perm in multiset_permutations(coords.T.tolist()):
                            perm_coords = [list(i) for i in zip(*perm)]
                            perm_coords.sort()
                            coords_to_check.add(tuple(map(tuple, perm_coords)))
                else:
                    coords = frozen_comp
                    for perm in multiset_permutations(coords.T.tolist()):
                        perm_coords = [list(i) for i in zip(*perm)]
                        perm_coords.sort()
                        coords_to_check.add(tuple(map(tuple, perm_coords)))
                        
        for coords in coords_to_check:
            total = 0
            for tile in coords:
                total += self.board[tile]
            if abs(total) == self.size:
                if total > 0:
                    score_p1 += 1
                else:
                    score_p2 += 1
                
        return score_p1, score_p2

In [3]:
game = Game(None, None, 3, 3)
game.board[0, 2, 2] = -1
game.board[1, 1, 1] = -1
game.board[2, 0, 0] = -1

In [7]:
game = Game(None, None, 3, 9)
game.board[0, 0, 0, 0, 0, 0, 0, 2, 2] = -1
game.board[0, 0, 0, 0, 0, 0, 1, 1, 1] = -1
game.board[0, 0, 0, 0, 0, 0, 2, 0, 0] = -1
game.board[0, 0, 0, 0, 0, 0, 0, 0, 0] = 1
game.board[1, 1, 1, 1, 1, 1, 1, 1, 1] = 1
game.board[2, 2, 2, 2, 2, 2, 2, 2, 2] = 1

In [8]:
%time game.score()

Wall time: 24 s


(1, 1)